## Setup

In [50]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve, CalibrationDisplay
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
def create_pipeline(cat_features, num_features):
    pipe = Pipeline(
        [
            ("feature_engineering", ColumnTransformer([("ohe", OneHotEncoder(), cat_features), ("pt", "passthrough", num_features)])),
            ("model", model)
        ]
    )
    return pipe

## Reading in Data

In [40]:
df = pd.read_csv("data/modeling_data_2025.csv")
df = df[(df["type_abbreviation"] == "STD") & (df["start_game_seconds_remaining"] > 30)]
df.shape

(2128126, 37)

In [41]:
# Adding stats needed to reach double-digits and pace needed
tri_dbl_stats = ["pts", "treb", "ast", "stl", "blk"]
for stat in tri_dbl_stats:
    df[f"{stat}_needed"] = np.where(df[stat] >= 10, 0, 10 - df[stat])
    df[f"{stat}_needed_per_min"] = df[f"{stat}_needed"] / (df["start_game_seconds_remaining"] / 60)

In [47]:
train_games = np.random.choice(df["game_id"].unique(), size=1000)
train_df = df[df["game_id"].isin(train_games)].copy()
test_df = df[~df["game_id"].isin(train_games)].copy()

## EDA

## Modeling

In [43]:
model = LogisticRegression()

In [44]:
cat_features = ["home_away", "broadcast_market", "month"]
num_features = ["start_game_seconds_remaining", "score_margin", "pts_needed", "treb_needed", "ast_needed", "stl_needed", "blk_needed"]

pipe = create_pipeline(cat_features, num_features)

In [45]:
pipe.fit(train_df.drop("triple_double", axis=1), train_df["triple_double"])

C:\Users\Timo\anaconda3\envs\triple-double-predictor\lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,steps,"[('feature_engineering', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('ohe', ...), ('pt', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
